In [1]:
import pandas as pd
import numpy as np
import math
import pickle

In [2]:
stat2coord = pd.DataFrame(pd.read_csv('stat_2_coord.csv'))

In [3]:
stat2coord.columns = ['STATION', 'Lat', 'Lon']

In [4]:
stat2coord.head()

,STATION,Lat,Lon
0,103RD ST,40.796092,-73.961454
1,104TH ST-102ND ST,40.695178,-73.844330
2,104TH ST-OXFORD AV,40.681711,-73.837683
3,110TH ST,40.795020,-73.944250
4,110TH ST-CENTRAL PARK NORTH,40.799075,-73.951822


In [5]:
pickup = pd.DataFrame(pd.read_csv('uber-raw-data-apr14.csv'))

In [6]:
pickup['Closest_Station'] = ''
pickup['Closest_Distance'] = ''

In [7]:
pickup.head()

,Date/Time,Lat,Lon,Base,Closest_Station,Closest_Distance
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,,
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,,
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,,
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,,
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,,


In [8]:
%%time
station_name_list = stat2coord['STATION'].tolist()
station_lat_list = np.array(stat2coord['Lat'])
station_lon_list = np.array(stat2coord['Lon'])

CPU times: user 412 µs, sys: 57 µs, total: 469 µs
Wall time: 435 µs


In [9]:
len(station_name_list)

355

In [10]:
pickup_lat = np.array(pickup['Lat'])
pickup_lon = np.array(pickup['Lon'])
pickup_list = list(zip(pickup_lat, pickup_lon))

In [11]:
len(pickup_list)

564516

In [12]:
pickup_list[:5]

[(40.768999999999998, -73.954899999999995),
 (40.726700000000001, -74.034499999999994),
 (40.7316, -73.987300000000005),
 (40.758800000000001, -73.977599999999995),
 (40.759399999999999, -73.972200000000001)]

In [13]:
len(station_lat_list)

355

In [14]:
type(np.zeros(10))

numpy.ndarray

In [15]:
def find_closest_station(lat,lon,latlist,lonlist,namelist):
    
    latlat = np.zeros(len(latlist))
    latlat.fill(lat)
    
    lonlon = np.zeros(len(lonlist))
    lonlon.fill(lon)
    
    dist_list = ((latlist - latlat)*(latlist - latlat) + (lonlist - lonlon)*(lonlist - lonlon))**(0.5)
#    print(dist_list)
    min_ind = np.argmin(dist_list)
    #print(min_ind)
    closest_dist = dist_list[min_ind]
    closest_station = namelist[min_ind]
    return closest_station, closest_dist

In [16]:
%%time
# test on ONE (1) pickup

find_closest_station(40,-70,station_lat_list,station_lon_list,station_name_list)

CPU times: user 66 µs, sys: 27 µs, total: 93 µs
Wall time: 88.9 µs


('FAR ROCKAWAY-MOTT AV', 3.8036662148577225)

In [17]:
test_pickup_list = pickup_list[:1000]

In [18]:
%%time
# test on 1000 pickups

closest_station_list = []
closest_dist_list = []

for pl in test_pickup_list:
    cs, cd = find_closest_station(pl[0],pl[1],station_lat_list,station_lon_list,station_name_list)
    closest_station_list.append(cs)
    closest_dist_list.append(cd)
    

CPU times: user 14.3 ms, sys: 1.37 ms, total: 15.7 ms
Wall time: 14.5 ms


In [19]:
%%time
# on all pickups (0.5M)
closest_station_list = []
closest_dist_list = []

for pl in pickup_list:
    cs, cd = find_closest_station(pl[0],pl[1],station_lat_list,station_lon_list,station_name_list)
    closest_station_list.append(cs)
    closest_dist_list.append(cd)

CPU times: user 7.27 s, sys: 16.3 ms, total: 7.29 s
Wall time: 7.29 s


In [20]:
pickup.Closest_Station = closest_station_list
pickup.Closest_Distance = closest_dist_list

In [21]:
# show results: added two columns: Closest_Station and Closest_Distance
pickup.head()

,Date/Time,Lat,Lon,Base,Closest_Station,Closest_Distance
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,68TH ST-HUNTER COLLEGE,0.009011
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,CHAMBERS ST,0.027617
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,3RD AV,0.001717
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,5TH AV-53RD ST,0.002741
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,51ST ST,0.002310


In [22]:
with open('UberStationPickups.pickle', 'w') as file:
    pickle.dump([pickup], file)